In [14]:
# !pip install -U --user /kaggle/input/lama-nn/LightAutoML-0.3.7.4-py3-none-any.whl >> none

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import copy
import gc
import os
import time
import lightautoml

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# target = pd.read_csv("tr.csv", usecols=['user_id', 'is_male'])
# # test = pd.read_csv("/kaggle/input/mlcupmlfeaturesv6/test_ml_num_v2.csv", nrows=10000)
# # test.sort_values("user_id", inplace=True)

# print(len(target))

In [4]:
# oof_age = pd.read_csv("oof_warm_male_big_automl.csv")
# test_age = pd.read_csv("test_warm_male_big_automl.csv")

# pca_train = pd.read_csv("pca_train.csv")
# pca_test = pd.read_csv("pca_test.csv")

# train_dart_pred = pd.read_csv("train_dart_pred.csv")
# test_dart_pred = pd.read_csv("test_dart_pred.csv")

# fl_train = pd.read_csv("fl_train_11.csv")
# fl_test = pd.read_csv("fl_test_11.csv")

# gru_train = pd.read_csv("ismale_gru_oof.csv")
# gru_test = pd.read_csv("ismale_gru_test.csv")
# gru_train.rename(columns={'is_male':'is_male_gru'}, inplace=True)
# gru_test.rename(columns={'is_male':'is_male_gru'}, inplace=True)

# gru_train_2 = pd.read_csv("ismale_gru_oof (1).csv")
# gru_test_2 = pd.read_csv("ismale_gru_test (1).csv")
# gru_train_2.rename(columns={'is_male':'is_male_gru_2'}, inplace=True)
# gru_test_2.rename(columns={'is_male':'is_male_gru_2'}, inplace=True)

# lstm_train = pd.read_csv("ismale_lstm_oof.csv")
# lstm_test = pd.read_csv("ismale_lstm_test.csv")
# lstm_train.rename(columns={'is_male':'is_male_lstm'}, inplace=True)
# lstm_test.rename(columns={'is_male':'is_male_lstm'}, inplace=True)

In [5]:
# oof_age = oof_age.merge(train_dart_pred, on='user_id')
# test_age = test_age.merge(test_dart_pred, on='user_id')

# oof_age = oof_age.merge(pca_train, on='user_id')
# test_age = test_age.merge(pca_test, on='user_id')

# oof_age = oof_age.merge(fl_train, on='user_id')
# test_age = test_age.merge(fl_test, on='user_id')

# oof_age = oof_age.merge(gru_train, on='user_id', how ='left')
# test_age = test_age.merge(gru_test, on='user_id', how ='left')

# oof_age = oof_age.merge(lstm_train, on='user_id', how ='left')
# test_age = test_age.merge(lstm_test, on='user_id', how ='left')

# oof_age = oof_age.merge(gru_train_2, on='user_id', how ='left')
# test_age = test_age.merge(gru_test_2, on='user_id', how ='left')

# oof_age.head()

In [6]:
# oof_age = oof_age.merge(target, on='user_id')
# oof_age = oof_age[~oof_age['is_male'].isna() &  oof_age['is_male'] != 'NA']
# print(len(oof_age), len(test_age))

In [3]:
# # oof_age.to_csv("b/oof_ismale_st_lvl2.csv", index=False)
# test_age.to_csv("b/test_ismale_st_lvl2.csv", index=False)

oof_age = pd.read_csv("warm_male/oof_ismale_st_lvl2_v2.csv").drop(columns=['Unnamed: 0'])
test_age = pd.read_csv("warm_male/test_ismale_st_lvl2_v2.csv").drop(columns=['Unnamed: 0'])

In [4]:
test_age.head()

,user_id,Lvl_0_Pipe_0_Mod_0_LinearL2,Lvl_0_Pipe_1_Mod_0_LightGBM,Lvl_0_Pipe_1_Mod_1_CatBoost,Lvl_0_Pipe_1_Mod_2_Tuned_CatBoost,Lvl_0_Pipe_2_Mod_0_TorchNN_0,Lvl_0_Pipe_2_Mod_1_Tuned_TorchNN_resnet_tuned_1,Lvl_0_Pipe_2_Mod_2_TorchNN_resnet_2,Lvl_0_Pipe_2_Mod_3_TorchNN_dense_3,Lvl_0_Pipe_2_Mod_4_TorchNN_denselight_4,...,lda_28,lda_29,lda_30,lda_31,ag_bct_lvl_1,is_male_gru,is_male_lstm,is_male_gru_2,is_male_logreg_x,is_male_logreg_y
0,6,0.144457,0.070597,0.154154,0.148786,0.053203,0.068951,0.072003,0.063822,0.053244,...,4.767207,1.859899,1.291734,-0.428038,0.070495,0.113913,0.185562,0.123123,0.223682,0.278538
1,7,0.895343,0.968588,0.932379,0.937524,0.895675,0.877100,0.877983,0.910450,0.921649,...,-0.953456,1.115388,2.353088,0.242621,0.195180,0.910243,0.856550,0.920224,0.728885,0.569218
2,9,0.169868,0.063449,0.133050,0.125611,0.104504,0.135914,0.109820,0.070905,0.082838,...,0.295128,0.048182,-0.890352,1.329456,0.038221,0.231063,0.262376,0.209344,0.263981,0.313131
3,10,0.011678,0.006453,0.015440,0.014498,0.019361,0.017235,0.015598,0.011829,0.008951,...,-1.936946,-2.677067,-2.592109,0.582448,0.032352,0.145292,0.098797,0.064308,0.152783,0.202301
4,11,0.884761,0.954797,0.861269,0.878886,0.916013,0.932043,0.824351,0.894256,0.919443,...,-0.465922,0.255851,-0.311392,0.174629,0.956838,0.902929,0.842848,0.921871,0.778808,0.815141


In [5]:
# train = train[~train['is_male'].isna() & (train['is_male'].apply(lambda x: x != 'NA'))]
# train['is_male'] = train['is_male'].astype('int')
# print(len(train))

In [6]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
import time


roles = {'target': 'is_male',
                 'drop': ['user_id', 'index', "age"]}
default_nn_params = {
    "n_epochs": 200,
}
default_nn_params_2 = {
    "n_epochs": 100,
}
automl = TabularAutoML(
    task = Task(name='binary'),
    cpu_limit = 24,
    timeout=3600*1500,
    gpu_ids='0',
    debug=True,
    general_params = {"use_algos": [["linear_l2", "dense", "resnet", "lgb"],
                                    ["linear_l2", "denselight"]], "skip_conn": True},
    nn_pipeline_params = {"use_te": True,  "max_intersection_depth": 0},
    reader_params = {'cv': 10, 'random_state': 44},
)
start_time = time.time()
oof_pred = automl.fit_predict(
        oof_age,
        roles=roles,
        verbose = 1
    )
end_time = time.time()

[19:42:42] Stdout logging level is INFO.
[19:42:42] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[19:42:42] Task: binary

[19:42:42] Start automl preset with listed constraints:
[19:42:42] - time: 5400000.00 seconds
[19:42:42] - CPU: 24 cores
[19:42:42] - memory: 16 GB

[19:42:42] Train data shape: (257535, 51)

[19:42:45] Layer 1 train process start. Time left 5399996.74 secs
[19:42:45] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[19:42:53] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8976769157054519
[19:42:53] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[19:42:53] Time left 5399988.25 secs

[19:42:56] Selector_LightGBM fitting and predicting completed
[19:42:56] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[19:43:25] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8979532551525702
[19:43:25] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[19:43:25] Time left 5399956.47 secs

[1

In [7]:
print(f"spend time: {end_time - start_time}")
print(automl.create_model_str_desc())

spend time: 563.1602985858917
Models on level 0:
	 10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2
	 10 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM
	 10 averaged models Lvl_0_Pipe_2_Mod_0_TorchNN_dense_0
	 10 averaged models Lvl_0_Pipe_2_Mod_1_TorchNN_resnet_1

Final prediction for new objects (level 1) = 
	 0.87551 * (10 averaged models Lvl_1_Pipe_0_Mod_0_LinearL2) +
	 0.12449 * (10 averaged models Lvl_1_Pipe_1_Mod_0_TorchNN_denselight_0) 


In [14]:
test_pred_ismale = automl.predict(test_age)
sub = pd.DataFrame()
sub['user_id'] = test_age.user_id
sub['is_male'] = test_pred_ismale.data[: ,0]
sub.sort_values("user_id", inplace=True)
sub.to_csv("stack_lvl2_ismale_final.csv", index=False)

sub.head()

,user_id,is_male
0,6,0.070718
1,7,0.924408
2,9,0.082610
3,10,0.025988
4,11,0.945953


In [15]:
# 0.8985054971831067 final